In [ ]:
pip install wilds

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=4ff83991107f12c666af7f82d4a9a294b0baca946360fc9ebe0dd0a66f503e52
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 115.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 55.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 57.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-smh0tcyj
  Running command git clone --filter=blob:none --quiet https://github.com/py

In [ ]:
from torchtext.vocab import GloVe
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import os
import pandas as pd
import numpy as np
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe
import re
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
from wilds.common.grouper import CombinatorialGrouper
from wilds.common.utils import split_into_groups
from torch.autograd import grad
from wilds.common.data_loaders import get_eval_loader


class ToxicClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, embeddings_vectors, hidden_dim = 32, output_dim = 1):
        super(ToxicClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embeddings_vectors, freeze=True)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        _, embedded = self.rnn(embedded)
        return self.output(embedded[-1])

def tokenize(text, max_length = 100):
    tokenizer = get_tokenizer('basic_english')
    text = text.lower()
    text = re.sub(r"([.!?,'*])", r"", text)
    text = re.sub(r"([-])", r" ", text)
    tokens = tokenizer(text)
    if len(tokens) < max_length:
      tokens.extend(['<PAD>']*(max_length - len(tokens)))
    tokens = tokens[:max_length]
    tokens = [glove.stoi.get(token, len(glove.stoi) - 1) for token in tokens]
    tokens = np.array(tokens, dtype=np.int64)
    return tokens

def compute_irm_penalty(losses, dummy):
  g1 = grad(losses[0::2].mean(), dummy, create_graph=True)[0]
  g2 = grad(losses[1::2].mean(), dummy, create_graph=True)[0]
  return (g1 * g2).sum()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
token_dim = 50
length = 100

glove = GloVe(name='6B', dim=token_dim)
padding_vector = torch.zeros(token_dim)
padding_token = '<PAD>'
glove.itos.append(padding_token)  
glove.stoi[padding_token] = len(glove.itos) - 1 
glove.vectors = torch.cat((glove.vectors, padding_vector.unsqueeze(0)), dim=0) 

dataset = get_dataset(dataset="civilcomments", download=True)
train_data = dataset.get_subset(
    "train")
train_loader = get_train_loader("standard", train_data, batch_size=32)
test_data = dataset.get_subset(
    "val")
test_loader = get_train_loader("standard", train_data, batch_size=128)

identities = CombinatorialGrouper(dataset, [ 'female', 'black'])
train_loader = get_train_loader(
    "group", train_data, grouper=identities, n_groups_per_batch=4, batch_size=128)

model = ToxicClassifier(len(glove), token_dim, glove.vectors.to(device))
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr = 1e-1)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience = 10)

negative_samples = (train_data.y_array == 0).sum()
positive_samples = (train_data.y_array == 1).sum()
pos_weight = negative_samples / positive_samples
criterion = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
num_epochs = 20
for epoch in range(0, num_epochs):
  train_loss = 0.0
  train_correct = 0
  train_total = 0
  model.train()
  group_weights = torch.ones(256)
  group_weights = group_weights/group_weights.sum()
  group_weights = group_weights.to(device)
  for cur_train in tqdm(train_loader):
      dro_loss = 0
      input, label, metadata = cur_train
      idx, groups_idx, a = split_into_groups(identities.metadata_to_group(metadata))
      input = tuple(map(tokenize, input))
      input = torch.Tensor(input).long()
      group_loss = torch.zeros(4).to(device)
      cur_max = torch.zeros(1).to(device)
      for i, cur_idx in zip(groups_idx, idx):
        cur_in = torch.index_select(input, 0, i).to(device)
        cur_label = torch.index_select(label, 0 , i).to(device)
        output = model(cur_in).reshape(-1)
        predicted_labels = (torch.sigmoid(output) >= 0.5).float()
        group_loss[cur_idx] = criterion(output, cur_label.float())
        train_correct += (predicted_labels == cur_label).sum().item()
        train_total += len(cur_label)
        group_weights[i] = group_weights[i] * torch.exp(torch.Tensor([0.1]).to(device)*group_loss[cur_idx])
        if group_loss[cur_idx] > cur_max:
            cur_max = group_loss[cur_idx]
      group_weights = group_weights/group_weights.sum()
      optimizer.zero_grad()
      cur_max.backward()
      optimizer.step()

  train_loss /= len(train_loader.dataset)
  scheduler.step(train_loss)
  train_acc = train_correct / train_total
  print(f"Epoch {epoch+1}/{num_epochs},  Train Acc: {train_acc:.4f}")  

  test_loss = 0.0
  test_correct = 0
  test_total = 0
  pbar = tqdm(test_loader)
  model.eval()
  with torch.no_grad():
      for i, data in enumerate(pbar, 0):
          input, label, groupings = cur_train
          input = tuple(map(tokenize, input))
          input = torch.Tensor(input).long().to(device)
          label = label.to(device)
          output = model(input).reshape(-1)
          loss = criterion(output, label.float())
          predicted_labels = (torch.sigmoid(output) >= 0.5).float()
          test_total += len(label)
          test_correct += (predicted_labels == label).sum().item()
          test_loss += loss
          pbar.set_postfix(MSE=loss.item())
      
  test_loss /= len(test_loader.dataset)
  test_acc = test_correct / test_total
  print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")


test_data = dataset.get_subset(
    "test",
)
# Prepare the data loader
test_loader = get_eval_loader("standard", test_data, batch_size=32)
trues = []
preds = []
metadatas = []
for input, true, metadata in test_loader:
    with torch.no_grad():
      input = tuple(map(tokenize, input))
      input = torch.Tensor(input).long().to(device)
      output = model(input)
      output = (torch.sigmoid(output) >= 0.5).long().reshape(-1)
      trues.append(true.to('cpu'))
      preds.append(output.to('cpu'))
      metadatas.append(metadata.to('cpu'))
all_preds = torch.cat(preds, dim = 0)
all_trues = torch.cat(trues, dim = 0)
all_metas = torch.cat(metadatas, dim = 0)
print(dataset.eval(all_preds, all_trues, all_metas))

.vector_cache/glove.6B.zip: 862MB [02:43, 5.27MB/s]                           
100%|█████████▉| 399999/400000 [00:09<00:00, 41791.37it/s]


You can also download the dataset manually at https://wilds.stanford.edu/downloads.


90914816Byte [00:14, 6261213.62Byte/s]                            


Extracting data/civilcomments_v1.0/archive.tar.gz to data/civilcomments_v1.0

It took 0.27 minutes to download and uncompress the dataset.



  0%|          | 0/2101 [00:00<?, ?it/s]<ipython-input-3-c6d2e9f99458>:104: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  input = torch.Tensor(input).long()
100%|██████████| 2101/2101 [00:54<00:00, 38.35it/s]


...

Epoch 2/20,  Train Acc: 0.2563


100%|██████████| 2102/2102 [00:29<00:00, 71.96it/s, MSE=1.42]


Epoch 2/20, Test Loss: 0.0111, Test Acc: 0.2969


 60%|█████▉    | 1260/2101 [00:32<00:21, 38.96it/s]


KeyboardInterrupt: ignored